# Feed-Forward Model End-to-End Demo

This notebook demonstrates kerasfactory's BaseFeedForwardModel on a synthetic tabular regression task, including:

- Data generation and train/val/test split
- Model creation, training, and evaluation
- Plotly visualizations
- Model serialization and loading validation

## 1. Setup and Imports


In [1]:
import os
import tempfile
from typing import Any
import numpy as np
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import MeanAbsoluteError
from plotly.subplots import make_subplots
from kerasfactory.models.feed_forward import BaseFeedForwardModel

print("✅ All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

✅ All imports successful!
TensorFlow version: 2.18.0
Keras version: 3.8.0


## 2. Generate Synthetic Regression Data

We'll create a synthetic tabular dataset with 12 features and a continuous target.
The target will be a non-linear combination of features with noise.


In [2]:
print("📦 Generating synthetic data...")

# Reproducibility
np.random.seed(42)

num_samples = 2000
num_features = 12
feature_names = [f"feature_{i}" for i in range(num_features)]

# Generate correlated features
X = np.random.randn(num_samples, num_features).astype(np.float32)
X[:, 1] = 0.5 * X[:, 0] + 0.5 * X[:, 1]  # introduce correlation
X[:, 5] = np.sin(X[:, 2]) + 0.1 * np.random.randn(num_samples)

# Create a non-linear target with noise
true_weights = np.linspace(1.0, 0.2, num_features)
y = (
    2.0 * np.sin(X[:, 0])
    + 0.5 * X[:, 1] ** 2
    - 1.5 * X[:, 2]
    + (X[:, 3] * X[:, 4])
    + X @ true_weights
    + 0.5 * np.random.randn(num_samples)
).astype(np.float32)

# Train/Val/Test split
train_size = int(0.7 * num_samples)
val_size = int(0.15 * num_samples)

X_train = X[:train_size]
y_train = y[:train_size]

X_val = X[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]

X_test = X[train_size + val_size:]
y_test = y[train_size + val_size:]

print(f"✅ Data shapes -> Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")


📦 Generating synthetic data...
✅ Data shapes -> Train: (1400, 12), Val: (300, 12), Test: (300, 12)


## 3. Build Feed-Forward Model

We will use `BaseFeedForwardModel` with a few hidden layers. Inputs will be passed as a dict keyed by `feature_names` to demonstrate universal input handling.


In [3]:
# Prepare dict inputs expected by BaseFeedForwardModel when using multiple named inputs
X_train_dict = {name: X_train[:, i] for i, name in enumerate(feature_names)}
X_val_dict = {name: X_val[:, i] for i, name in enumerate(feature_names)}
X_test_dict = {name: X_test[:, i] for i, name in enumerate(feature_names)}

# Create model
model = BaseFeedForwardModel(
    feature_names=feature_names,
    hidden_units=[128, 64, 32],
    output_units=1,
    dropout_rate=0.2,
    activation='relu',
    name='feed_forward_demo'
)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=MeanSquaredError(),
    metrics=[MeanAbsoluteError()],
)

print("✅ Model created and compiled!")
print("Feature names:", feature_names)


2025-11-13 10:36:05.643 | INFO     | kerasfactory.models.feed_forward:__init__:84 - 🏗️ Initializing Feed Forward Neural Network
2025-11-13 10:36:05.644 | INFO     | kerasfactory.models.feed_forward:__init__:85 - 📊 Model Architecture: [128, 64, 32] -> 1
2025-11-13 10:36:05.645 | INFO     | kerasfactory.models.feed_forward:__init__:86 - 🔄 Input Features: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11']
2025-11-13 10:36:05.647 | DEBUG    | kerasfactory.models.feed_forward:__init__:92 - ✨ Created input layers for features: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11']
2025-11-13 10:36:05.649 | DEBUG    | kerasfactory.models.feed_forward:__init__:96 - ✨ Created concatenation layer
2025-11-13 10:36:05.649 | DEBUG    | kerasfactory.models.feed_forward:__init__:101 - ✨ Addin

✅ Model created and compiled!
Feature names: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11']


## 4. Train and Evaluate
We will train for a small number of epochs and then evaluate on the validation and test sets.


In [4]:
print("🚀 Starting training...")

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)
]

history = model.fit(
    X_train_dict, y_train,
    validation_data=(X_val_dict, y_val),
    epochs=30,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print("✅ Training completed!")

# Evaluate
val_loss, val_mae = model.evaluate(X_val_dict, y_val, verbose=0)
print(f"Validation - Loss: {val_loss:.4f}, MAE: {val_mae:.4f}")

test_loss, test_mae = model.evaluate(X_test_dict, y_test, verbose=0)
print(f"Test - Loss: {test_loss:.4f}, MAE: {test_mae:.4f}")

# Predictions for later visualizations
y_pred_val = model.predict(X_val_dict, verbose=0).squeeze()
y_pred_test = model.predict(X_test_dict, verbose=0).squeeze()


🚀 Starting training...
Epoch 1/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11.3193 - mean_absolute_error: 2.7358 - val_loss: 4.8158 - val_mean_absolute_error: 1.7568 - learning_rate: 0.0010
Epoch 2/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.0532 - mean_absolute_error: 1.5298 - val_loss: 1.5456 - val_mean_absolute_error: 0.9605 - learning_rate: 0.0010
Epoch 3/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3619 - mean_absolute_error: 1.1606 - val_loss: 1.4034 - val_mean_absolute_error: 0.9238 - learning_rate: 0.0010
Epoch 4/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.0067 - mean_absolute_error: 1.0602 - val_loss: 1.0851 - val_mean_absolute_error: 0.8248 - learning_rate: 0.0010
Epoch 5/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6823 - mean_absolute_error: 1.0041 - val_loss: 0.9858 - val_mean_absolute_error: 0.7758 - learning_rate: 0.0010
Epoch 6/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.3893 - mean_absolute_error: 0.8885 - val_loss: 0.8052 - 

## 5. Visualizations
We will plot training curves, predictions vs. ground truth, and residuals.


In [5]:
print("📊 Creating visualizations...")

# Loss curves
hist_loss = history.history.get("loss", [])
hist_val_loss = history.history.get("val_loss", [])

# Residuals (test)
residuals = (y_test - y_pred_test)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Training/Validation Loss",
        "Predictions vs Ground Truth (Test)",
        "Residuals Histogram (Test)",
        "MAE over Bins (Test)"
    ),
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "histogram"}, {"type": "bar"}]]
)

# Plot 1: Loss curves
fig.add_scatter(y=hist_loss, mode="lines", name="loss", row=1, col=1)
fig.add_scatter(y=hist_val_loss, mode="lines", name="val_loss", row=1, col=1)

# Plot 2: Predictions vs truth (downsample for display if large)
idx = np.random.choice(len(y_test), size=min(400, len(y_test)), replace=False)
fig.add_scatter(
    x=y_test[idx], y=y_pred_test[idx], mode="markers", name="pred vs true",
    row=1, col=2
)
# Diagonal reference line
min_v = float(min(y_test.min(), y_pred_test.min()))
max_v = float(max(y_test.max(), y_pred_test.max()))
fig.add_scatter(x=[min_v, max_v], y=[min_v, max_v], mode="lines", name="ideal", row=1, col=2)

# Plot 3: Residuals histogram
fig.add_histogram(x=residuals, nbinsx=40, name="residuals", row=2, col=1)

# Plot 4: MAE over bins
bins = np.linspace(min_v, max_v, 20)
bin_indices = np.digitize(y_test, bins)
mae_per_bin = []
centers = []
for b in range(1, len(bins)):
    mask = bin_indices == b
    if np.any(mask):
        mae_per_bin.append(np.mean(np.abs(y_test[mask] - y_pred_test[mask])))
        centers.append((bins[b] + bins[b-1]) / 2)
    else:
        mae_per_bin.append(0.0)
        centers.append((bins[b] + bins[b-1]) / 2)

fig.add_bar(x=centers, y=mae_per_bin, name="MAE per bin", row=2, col=2)

fig.update_layout(height=800, title_text="Feed-Forward Regression Results", showlegend=True)
fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_xaxes(title_text="True", row=1, col=2)
fig.update_yaxes(title_text="Predicted", row=1, col=2)
fig.update_xaxes(title_text="Residual", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_xaxes(title_text="Target (bin center)", row=2, col=2)
fig.update_yaxes(title_text="MAE", row=2, col=2)

fig.show()
print("✅ Visualizations created successfully!")


📊 Creating visualizations...


✅ Visualizations created successfully!


## 6. Model Serialization and Loading
We will save the model in Keras format and validate that a loaded model produces consistent predictions.


In [6]:
print("💾 Testing Keras format serialization...")

with tempfile.TemporaryDirectory() as temp_dir:
    keras_path = os.path.join(temp_dir, "feed_forward_demo.keras")

    # Save
    model.save(keras_path)
    print(f"✅ Model saved to: {keras_path}")

    # Load
    loaded_model = keras.models.load_model(keras_path)
    print("✅ Model loaded successfully!")

    # Compare predictions on a small slice
    sl = slice(0, 64)
    preds_orig = model.predict({k: v[sl] for k, v in X_test_dict.items()}, verbose=0)
    preds_loaded = loaded_model.predict({k: v[sl] for k, v in X_test_dict.items()}, verbose=0)

    # Report similarity
    diff = np.mean(np.abs(preds_orig - preds_loaded))
    print(f"🔍 Mean absolute difference between original and loaded predictions: {float(diff):.6f}")
    print("✅ Loaded model prediction check completed!")


2025-11-13 10:36:09.426 | INFO     | kerasfactory.models.feed_forward:__init__:84 - 🏗️ Initializing Feed Forward Neural Network
2025-11-13 10:36:09.427 | INFO     | kerasfactory.models.feed_forward:__init__:85 - 📊 Model Architecture: [128, 64, 32] -> 1
2025-11-13 10:36:09.427 | INFO     | kerasfactory.models.feed_forward:__init__:86 - 🔄 Input Features: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11']
2025-11-13 10:36:09.429 | DEBUG    | kerasfactory.models.feed_forward:__init__:92 - ✨ Created input layers for features: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11']
2025-11-13 10:36:09.430 | DEBUG    | kerasfactory.models.feed_forward:__init__:96 - ✨ Created concatenation layer
2025-11-13 10:36:09.430 | DEBUG    | kerasfactory.models.feed_forward:__init__:101 - ✨ Addin

💾 Testing Keras format serialization...
✅ Model saved to: /var/folders/v8/4l9cyywn1x970gdc1v67r5480000gn/T/tmp4itjkks_/feed_forward_demo.keras
✅ Model loaded successfully!
🔍 Mean absolute difference between original and loaded predictions: 0.000000
✅ Loaded model prediction check completed!


## 7. Summary

- Trained `BaseFeedForwardModel` on synthetic regression data with 12 features.
- Achieved regression metrics (reported above) on validation and test sets.
- Visualized loss curves, predictions vs. truth, and residuals.
- Saved and loaded model in Keras format; verified prediction consistency.

This notebook mirrors the style of the autoencoder end-to-end test and showcases the feed-forward workflow.
